In [25]:
import mlrose_hiive
import numpy as np

from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt

import mlrose_hiive as mlrose
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from mlrose_hiive import MaxKColorGenerator, QueensGenerator, FlipFlopGenerator,\
      TSPGenerator, KnapsackGenerator, ContinuousPeaksGenerator
from mlrose_hiive import SARunner, GARunner, NNGSRunner, MIMICRunner, RHCRunner
from mlrose_hiive import SKMLPRunner

In [26]:
mlrose_hiive.NeuralNetwork(hidden_nodes = [], activation = 'sigmoid', 
                                    algorithm = 'random_hill_climb', 
                                    max_iters = 1000, bias = True, is_classifier = True, 
                                    learning_rate = 0.01, early_stopping = True, 
                                    clip_max = 5, max_attempts = 100, random_state = 3)

NeuralNetwork(activation='sigmoid', clip_max=5, early_stopping=True,
              hidden_nodes=[], learning_rate=0.01, max_attempts=100,
              max_iters=1000, random_state=3)

In [35]:
problem_size = [10, 20, 40, 60, 100]#, 500, 1000] 500 error out
flip_problem = FlipFlopGenerator().generate(size=20,seed=123456)
queen_problem = QueensGenerator().generate(size=20,seed=123456)
knap_problem = KnapsackGenerator().generate(number_of_items_types=10,seed=123456)

In [30]:
# create a runner class and solve the problem
rhc_run = RHCRunner(problem=queen_problem,
              experiment_name='queens8_sa',
              output_directory=None, # note: specify an output directory to have results saved to disk
              seed=123456,
              iteration_list=2 ** np.arange(11),
              max_attempts=500,
              restart_list=[20, 40, 60]
              )

# the two data frames will contain the results
df_run_stats, df_run_curves = rhc_run.run()

In [33]:
df_run_curves[(df_run_curves['Fitness']==0)]

,Iteration,Time,Fitness,FEvals,Restarts,max_iters,current_restart
19916,519,0.319178,0.0,10387.0,20,1024,11


In [11]:
# create a runner class and solve the problem
sa_run = SARunner(problem=knap_problem,
              experiment_name='queens8_sa',
              output_directory=None, # note: specify an output directory to have results saved to disk
              seed=123456,
              iteration_list=2 ** np.arange(11),
              max_attempts=500,
              temperature_list=[0.1, 0.5, 0.75, 1.0, 2.0, 5.0],
              decay_list=[mlrose.GeomDecay])

# the two data frames will contain the results
df_run_stats, df_run_curves = sa_run.run()

In [40]:
def run_rhc_optimization(problem):
    # create a runner class and solve the problem
    rhc_run = RHCRunner(problem=problem,
                experiment_name='queens8_sa',
                output_directory=None, # note: specify an output directory to have results saved to disk
                seed=123456,
                iteration_list=2 ** np.arange(11),
                max_attempts=500,
                restart_list=[25, 75, 100]
                )

    # the two data frames will contain the results
    df_run_stats, df_run_curves = rhc_run.run()
    name = rhc_run.runner_name()
    return df_run_stats, df_run_curves, name

def run_ga_optimization(problem):
    # create a runner class and solve the problem
    ga_run = GARunner(problem=problem,
                experiment_name='queens8_sa',
                output_directory=None, # note: specify an output directory to have results saved to disk
                seed=123456,
                iteration_list=2 ** np.arange(11),
                max_attempts=500,
                population_sizes = [200, 400, 600],
                mutation_rates = [0.25, 0.5, 0.75]
                )

    # the two data frames will contain the results
    df_run_stats, df_run_curves = ga_run.run()
    name = ga_run.runner_name()
    return df_run_stats, df_run_curves, name

def run_sa_optimization(problem):
    # create a runner class and solve the problem
    sa_run = SARunner(problem=problem,
                experiment_name='queens8_sa',
                output_directory=None, # note: specify an output directory to have results saved to disk
                seed=123456,
                iteration_list=2 ** np.arange(11),
                max_attempts=500,
                temperature_list=[0.1, 0.5, 0.75, 1.0, 2.0, 5.0],
                decay_list=[mlrose.GeomDecay])

    # the two data frames will contain the results
    df_run_stats, df_run_curves = sa_run.run()
    name = sa_run.runner_name()
    return df_run_stats, df_run_curves, name


def run_mimic_optimization(problem):
    # create a runner class and solve the problem
    mimic_run = MIMICRunner(problem=problem,
                experiment_name='queens8_sa',
                output_directory=None, # note: specify an output directory to have results saved to disk
                seed=123456,
                iteration_list=2 ** np.arange(11),
                max_attempts=500,
                population_sizes = [200, 400, 600],
                keep_percent_list = [0.25, 0.5, 0.75]
                )

    # the two data frames will contain the results
    df_run_stats, df_run_curves = mimic_run.run()
    name = mimic_run.runner_name()
    return df_run_stats, df_run_curves, name


In [41]:
count = 1
for size in problem_size:
    flip_problem = FlipFlopGenerator().generate(size=size,seed=123456)
    queen_problem = QueensGenerator().generate(size=size,seed=123456)
    knap_problem = KnapsackGenerator().generate(number_of_items_types=size,seed=123456)
    flip_string = 'flip_flop'
    queen_string = 'queen'
    knap_string = 'knap'
    problem_lst = [(flip_problem, flip_string), (queen_problem,queen_string),\
                    (knap_problem, knap_string)]
    algo_string = f'mimic_results_{count}.csv'

    for problem in problem_lst:
        opt_problem = problem[0]
        problem_name = problem[1]

        # run experiments for randomized hill climbing
        df_stats, df_curves, name = run_rhc_optimization(problem=opt_problem)
        df_stats.to_csv(f'{name}_stats_{problem_name}_size_{size}_iteration_{count}.csv')
        df_curves.to_csv(f'{name}_curves_{problem_name}_size_{size}_iteration_{count}.csv')
        
        # run experiments for genetic algorithms
        df_stats, df_curves, name = run_ga_optimization(problem=opt_problem)
        df_stats.to_csv(f'{name}_stats_{problem_name}_size_{size}_iteration_{count}.csv')
        df_curves.to_csv(f'{name}_curves_{problem_name}_size_{size}_iteration_{count}.csv')
        
        # run experiments for simulated annealing
        df_stats, df_curves, name = run_sa_optimization(problem=opt_problem)
        df_stats.to_csv(f'{name}_stats_{problem_name}_size_{size}_iteration_{count}.csv')
        df_curves.to_csv(f'{name}_curves_{problem_name}_size_{size}_iteration_{count}.csv')
        
        # run experiments for mimic
        df_stats, df_curves, name = run_mimic_optimization(problem=opt_problem)
        df_stats.to_csv(f'{name}_stats_{problem_name}_size_{size}_iteration_{count}.csv')
        df_curves.to_csv(f'{name}_curves_{problem_name}_size_{size}_iteration_{count}.csv')

    count = count + 1

MemoryError: Unable to allocate 954. MiB for an array with shape (500, 500, 500) and data type float64